In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split


In [8]:
train_df = pd.read_csv(
    "train.csv",
    header=None,
    names=['label', 'title', 'description'],
    engine='python',
    on_bad_lines='skip'
)

test_df = pd.read_csv("test.csv", header=None, names=['label', 'title', 'description'])

train_df['label'] -= 1
test_df['label'] -= 1


train_df['text'] = train_df['title'] + '. ' + train_df['description']
test_df['text'] = test_df['title'] + '. ' + test_df['description']


train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'].tolist(),
    train_df['label'].tolist(),
    test_size=0.1,
    random_state=42
)


In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch_texts):
    return tokenizer(batch_texts, padding=True, truncation=True, max_length=256, return_tensors="pt")

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
test_encodings = tokenize(test_df['text'].tolist())



In [10]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(
train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)



In [11]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4
)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_steps=200,
    save_steps=200,
    logging_dir='./logs',
    load_best_model_at_end=True,
     fp16=True,
    eval_strategy="steps"  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
200,No log,0.293567
400,No log,0.312086
600,0.377800,0.278156
800,0.377800,0.267991
1000,0.283400,0.246954
1200,0.283400,0.255717
1400,0.283400,0.237386
1600,0.270400,0.234821
1800,0.270400,0.229576
2000,0.251200,0.220812


TrainOutput(global_step=6750, training_loss=0.22926017422146266, metrics={'train_runtime': 1879.6003, 'train_samples_per_second': 57.459, 'train_steps_per_second': 3.591, 'total_flos': 7153494663168000.0, 'train_loss': 0.22926017422146266, 'epoch': 1.0})

In [12]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

trainer.compute_metrics = compute_metrics
eval_results = trainer.evaluate()
print(eval_results)






{'eval_loss': 0.1673486977815628, 'eval_accuracy': 0.946, 'eval_f1': 0.9460019441862085, 'eval_runtime': 20.3933, 'eval_samples_per_second': 588.429, 'eval_steps_per_second': 36.777, 'epoch': 1.0}


In [13]:
# Save
model.save_pretrained("news_topic_model")
tokenizer.save_pretrained("news_topic_model")


('news_topic_model/tokenizer_config.json',
 'news_topic_model/special_tokens_map.json',
 'news_topic_model/vocab.txt',
 'news_topic_model/added_tokens.json')

In [14]:
import gradio as gr

def predict_news_topic(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred_class = torch.argmax(probs).item()
    return f"Predicted Topic: {pred_class} | Probabilities: {probs.tolist()}"

gr.Interface(
    fn=predict_news_topic,
    inputs=gr.Textbox(lines=3, placeholder="Enter news headline here..."),
    outputs="text",
    title="News Topic Classifier",
    description="Classify news headline into topic using fine-tuned BERT"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1537e9ef3b9092155.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!cp -r news_topic_model /content/drive/MyDrive/

In [17]:
!cp -r results/ /content/drive/MyDrive/

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

In [18]:
!cp -r wandb/ /content/drive/MyDrive/